# Results Analysis
__authors__: ufukbombar@gmail.com

__date__: 2024-11-18

__inputs__:
 - Direct query to the clickhouse
 - Measurements folder

__goal__: This notebook downloads the results table

In [109]:
# Required packages
%pip install pandas requests python-dotenv httpx aiofiles dataclasses tqdm pyarrow

# External imports
import pandas as pd 
from dataclasses import dataclass
import json
from datetime import datetime
from tqdm.asyncio import tqdm
import os

# Internal imports
from config import config
import common
import file as ff

Note: you may need to restart the kernel to use updated packages.


In [114]:
# This is the output of table_downloader
input_measurement_name = "../data/measurements/2024-11-18 20:37:25.feather"

# Set this to True to redo all the work, else loads from existing data.
output_measurement_name = None
output_measurement_name = "../data/results/results-limit1000002024-11-19 00:25:08007046a9_518e_46cb_8e70_e598b8bce831.feather" 

In [115]:
input_measurement_df = pd.read_feather("../data/measurements/measurement-2024-11-18 22:44:16.feather")
input_measurement_df

,measurement_uuid,type,cleaned,prefix,agent_uuid,table_name,retrieved,processed
0,007046a9_518e_46cb_8e70_e598b8bce831,links,True,cleaned_links,400a3c9b_57ed_4315_9489_917e601f3604,cleaned_links__007046a9_518e_46cb_8e70_e598b8b...,2024-11-18 20:10:38.865040,2024-11-18 22:44:14.561684
1,007046a9_518e_46cb_8e70_e598b8bce831,links,True,cleaned_links,4f6137c8_dfc5_4043_8005_d617407b7be3,cleaned_links__007046a9_518e_46cb_8e70_e598b8b...,2024-11-18 20:10:38.865040,2024-11-18 22:44:14.561735
2,007046a9_518e_46cb_8e70_e598b8bce831,links,True,cleaned_links,51fb56f4_5217_4108_a212_2d05806b16dc,cleaned_links__007046a9_518e_46cb_8e70_e598b8b...,2024-11-18 20:10:38.865040,2024-11-18 22:44:14.561759
3,007046a9_518e_46cb_8e70_e598b8bce831,links,True,cleaned_links,5bdab212_3d82_4eca_8c5a_39e1c8ded842,cleaned_links__007046a9_518e_46cb_8e70_e598b8b...,2024-11-18 20:10:38.865040,2024-11-18 22:44:14.561786
4,007046a9_518e_46cb_8e70_e598b8bce831,links,True,cleaned_links,70ce196e_9cb7_4467_b1a8_c40400f6f5df,cleaned_links__007046a9_518e_46cb_8e70_e598b8b...,2024-11-18 20:10:38.865040,2024-11-18 22:44:14.561816
...,...,...,...,...,...,...,...,...
100548,ffd2d86c_05c1_4607_a5b1_4ab04b77ee41,results,False,results,e05ddb8f_c572_4d28_9ffb_8e24bbf2027a,results__ffd2d86c_05c1_4607_a5b1_4ab04b77ee41_...,2024-11-18 20:10:38.865040,2024-11-18 22:44:15.803084
100549,ffd9d901_2127_4048_8f9a_805e3fa4930c,results,False,results,ddd8541d_b4f5_42ce_b163_e3e9bfcd0a47,results__ffd9d901_2127_4048_8f9a_805e3fa4930c_...,2024-11-18 20:10:38.865040,2024-11-18 22:44:15.803095
100550,ffebc346_5c58_4d73_8866_1746dedadc50,results,False,results,ddd8541d_b4f5_42ce_b163_e3e9bfcd0a47,results__ffebc346_5c58_4d73_8866_1746dedadc50_...,2024-11-18 20:10:38.865040,2024-11-18 22:44:15.803107
100551,ffec2848_65c7_44d1_8f15_8bd08e2d41f0,results,False,results,ddd8541d_b4f5_42ce_b163_e3e9bfcd0a47,results__ffec2848_65c7_44d1_8f15_8bd08e2d41f0_...,2024-11-18 20:10:38.865040,2024-11-18 22:44:15.803118


In [116]:
picked_measurement_row = input_measurement_df[input_measurement_df['cleaned'] == True].iloc[0]
picked_measurement_uuid = picked_measurement_row['measurement_uuid']
limit_results_table = 10000

print("Number of measurements", len(input_measurement_df['measurement_uuid'].unique()))

print("Picked measurement", picked_measurement_uuid)
print("Picked tables")
set(input_measurement_df[
    (input_measurement_df["measurement_uuid"] == picked_measurement_uuid) & 
    (input_measurement_df["cleaned"] == True) & 
    (input_measurement_df["type"] == "results")]['table_name'])

Number of measurements 6903
Picked measurement 007046a9_518e_46cb_8e70_e598b8bce831
Picked tables


{'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00d',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__bc17c7f9_9d02_48de_b766_8c927c8c10fd',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__ccdf7970_8877_42cc_960d_41d0d0d0c302',
 'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__f39c7a18_5b00_4e57_9694_f3db8198b72a'}

In [117]:
def construct_sql_statement(df: pd.DataFrame, measurement_uuid: str, table_type: str="results", cleaned_only: bool=True, limit: int=1):
    df_trimmed = df[(df["measurement_uuid"] == measurement_uuid) &
                         (df["type"] == table_type) &
                         (df["cleaned"] == cleaned_only)]
    
    limit_str = "" if limit == 0 else f"limit {limit}"
    query_parts = [f"select * from {table} {limit_str}" for table in df_trimmed['table_name']]
    combined_query = " union all ".join(query_parts)
    return combined_query

async def download_joined_results(limit: int):
    query_str = construct_sql_statement(input_measurement_df, picked_measurement_uuid, limit=limit)

    with ff.TemporaryFile() as temp:
        with tqdm(desc="Running the query result", unit="MB") as pb:
            async for chunk in common.run_query(cfg=config, query=query_str):
                temp.write(chunk)
                pb.update(len(chunk) // config.chunk_size)
        temp.close()
        return pd.read_csv(temp.filename)

if not output_measurement_name: 
    result_table_df = await download_joined_results(limit_results_table)
    result_table_df
    # ff.save_dataframe_feather(result_table_df, "../data/results", prefix=f"results-limit{limit_results_table}", postfix=picked_measurement_uuid)
else:
    result_table_df = pd.read_feather(output_measurement_name)

result_table_df

,capture_timestamp,probe_protocol,probe_src_addr,probe_dst_addr,probe_src_port,probe_dst_port,probe_ttl,quoted_ttl,reply_src_addr,reply_protocol,...,rtt,round,probe_dst_prefix,reply_src_prefix,private_probe_dst_prefix,private_reply_src_addr,destination_host_reply,destination_prefix_reply,valid_probe_protocol,time_exceeded_reply
0,2024-05-22 02:45:08,1,::ffff:10.0.0.2,::ffff:119.219.98.255,24109,0,6,1,::ffff:112.174.25.122,1,...,504,4,::ffff:119.219.98.0,::ffff:112.174.25.0,0,0,0,0,1,1
1,2024-05-22 02:45:08,1,::ffff:10.0.0.2,::ffff:119.219.98.255,24109,0,7,1,::ffff:125.144.27.222,1,...,567,4,::ffff:119.219.98.0,::ffff:125.144.27.0,0,0,0,0,1,1
2,2024-05-22 02:45:08,1,::ffff:10.0.0.2,::ffff:119.219.98.255,24109,0,8,1,::ffff:125.145.101.142,1,...,609,4,::ffff:119.219.98.0,::ffff:125.145.101.0,0,0,0,0,1,1
3,2024-05-22 02:47:05,1,::ffff:10.0.0.2,::ffff:119.219.98.255,24110,0,6,1,::ffff:125.144.21.110,1,...,647,4,::ffff:119.219.98.0,::ffff:125.144.21.0,0,0,0,0,1,1
4,2024-05-22 02:47:05,1,::ffff:10.0.0.2,::ffff:119.219.98.255,24110,0,7,1,::ffff:125.145.101.142,1,...,614,4,::ffff:119.219.98.0,::ffff:125.145.101.0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2024-05-22 02:50:52,1,::ffff:10.0.6.2,::ffff:165.204.86.255,26402,0,7,1,::ffff:172.71.185.33,1,...,1090,4,::ffff:165.204.86.0,::ffff:172.71.185.0,0,0,0,0,1,1
999996,2024-05-22 02:51:27,1,::ffff:10.0.6.2,::ffff:165.204.86.255,26405,0,6,1,::ffff:172.71.185.32,1,...,1090,4,::ffff:165.204.86.0,::ffff:172.71.185.0,0,0,0,0,1,1
999997,2024-05-22 02:51:27,1,::ffff:10.0.6.2,::ffff:165.204.86.255,26405,0,7,1,::ffff:172.71.185.48,1,...,1092,4,::ffff:165.204.86.0,::ffff:172.71.185.0,0,0,0,0,1,1
999998,2024-05-22 02:53:10,1,::ffff:10.0.6.2,::ffff:165.204.86.255,26406,0,6,1,::ffff:172.71.185.125,1,...,1089,4,::ffff:165.204.86.0,::ffff:172.71.185.0,0,0,0,0,1,1


In [118]:
print("Set of all columns")

{'capture_timestamp',
 'destination_host_reply',
 'destination_prefix_reply',
 'private_probe_dst_prefix',
 'private_reply_src_addr',
 'probe_dst_addr',
 'probe_dst_port',
 'probe_dst_prefix',
 'probe_protocol',
 'probe_src_addr',
 'probe_src_port',
 'probe_ttl',
 'quoted_ttl',
 'reply_icmp_code',
 'reply_icmp_type',
 'reply_mpls_labels',
 'reply_protocol',
 'reply_size',
 'reply_src_addr',
 'reply_src_prefix',
 'reply_ttl',
 'round',
 'rtt',
 'time_exceeded_reply',
 'valid_probe_protocol'}

Set of all columns


{'capture_timestamp',
 'destination_host_reply',
 'destination_prefix_reply',
 'private_probe_dst_prefix',
 'private_reply_src_addr',
 'probe_dst_addr',
 'probe_dst_port',
 'probe_dst_prefix',
 'probe_protocol',
 'probe_src_addr',
 'probe_src_port',
 'probe_ttl',
 'quoted_ttl',
 'reply_icmp_code',
 'reply_icmp_type',
 'reply_mpls_labels',
 'reply_protocol',
 'reply_size',
 'reply_src_addr',
 'reply_src_prefix',
 'reply_ttl',
 'round',
 'rtt',
 'time_exceeded_reply',
 'valid_probe_protocol'}

In [119]:
print("Here are the required column names")
['probe_dst_prefix', 'reply_src_prefix']

Here are the required column names


['probe_dst_prefix', 'reply_src_prefix']

# DO NOT RUN HERE

In [120]:
async def count_joined_results(additionalt_query: str=""):
    query_str = construct_sql_statement(input_measurement_df, picked_measurement_uuid, limit=0)
    query_str = f"select count(*) from ({query_str}) as combined_tables {additionalt_query}"

    with ff.TemporaryFile() as temp:
        with tqdm(desc="Counting the query result", unit="MB") as pb:
            async for chunk in common.run_query(cfg=config, query=query_str):
                temp.write(chunk)
                pb.update(len(chunk) // config.chunk_size)
        temp.close()
        return pd.read_csv(temp.filename)['count()'].item()

count_results_table = await count_joined_results()
print(f"There is a total of {count_results_table} rows in one measurement with IP address starts with ':ffff:' (all results tables combined)")

Counting the query result: 0MB [00:00, ?MB/s]

There is a total of 758301083 rows in one measurement with IP address starts with ':ffff:' (all results tables combined)


In [72]:
print(f"In case of a 2-tuple compression (remove after /24 so 2 IPv4 addresses would take 6 bytes) there will be a maximum of {count_results_table * 6}")

In case of a 2-tuple compression (remove after /24 so 2 IPv4 addresses would take 6 bytes) there will be a maximum of 4549806498


In [73]:
for i, measure in enumerate(['bytes', 'Kb', 'Mb', 'Gb']):
    print(f"Which means the binary file would be (maximum of) {count_results_table * 6 / (1024 ** i):.2f} {measure}")

Which means the binary file would be (maximum of) 4549806498.00 bytes
Which means the binary file would be (maximum of) 4443170.41 Kb
Which means the binary file would be (maximum of) 4339.03 Mb
Which means the binary file would be (maximum of) 4.24 Gb


In [ ]:
async def get_ip_address_as_number(meas_uuid: str=""):
    query_str = f"select IPv4StringToNum(IPv6NumToIPv4(probe_dst_prefix)) AS probe_dst_prefix_number from {meas_uuid};"

    with ff.TemporaryFile() as temp:
        with tqdm(desc="Counting the query result", unit="MB") as pb:
            async for chunk in common.run_query(cfg=config, query=query_str):
                temp.write(chunk)
                pb.update(len(chunk) // config.chunk_size)
        temp.close()
        return pd.read_csv(temp.filename)['count()'].item()

count_results_table = await get_ip_address_as_number(picked_measurement_uuid)
print(f"There is a total of {count_results_table} rows in one measurement with IP address starts with ':ffff:' (all results tables combined)")

Counting the query result: 0MB [00:00, ?MB/s]

An exception occurred: <class 'httpx.HTTPStatusError'>, Client error '404 Not Found' for url 'https://chproxy.iris.dioptra.io/?enable_http_compression=false&default_format=CSVWithNames&output_format_json_quote_64bit_integer=&database=iris&query=select%20IPv4StringToNum%28IPv6NumToIPv4%28probe_dst_prefix%29%29%20AS%20probe_dst_prefix_number%20from%20007046a9_518e_46cb_8e70_e598b8bce831%3B'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404
There is a total of None rows in one measurement with IP address starts with ':ffff:' (all results tables combined)


In [129]:
async def test_query():
    # q = "WITH IPv6StringToNum(address_raw) as address_raw SELECT cutIPv6(address_raw, 0, 0)"
    # q = "SELECT IPv6NumToString(toFixedString(unhex('2A0206B8000000000000000000000011'), 16)) AS addr;"
    q = "SELECT addr, cutIPv6(IPv6StringToNum(addr), 0, 0) FROM (SELECT ['notaddress', '127.0.0.1', '1111::ffff'] AS addr) ARRAY JOIN addr;"
    with ff.TemporaryFile() as temp:
        with tqdm(desc="Running the query result", unit="MB") as pb:
            async for chunk in common.run_query(cfg=config, query=q):
                temp.write(chunk)
                pb.update(len(chunk) // config.chunk_size)
        temp.close()
        return pd.read_csv(temp.filename)
    
test_df = await test_query()
test_df

Running the query result: 0MB [00:00, ?MB/s]

An exception occurred: <class 'httpx.HTTPStatusError'>, Server error '500 Internal Server Error' for url 'https://chproxy.iris.dioptra.io/?enable_http_compression=false&default_format=CSVWithNames&output_format_json_quote_64bit_integer=&database=iris&query=SELECT%20addr%2C%20cutIPv6%28IPv6StringToNum%28addr%29%2C%200%2C%200%29%20FROM%20%28SELECT%20%5B%27notaddress%27%2C%20%27127.0.0.1%27%2C%20%271111%3A%3Affff%27%5D%20AS%20addr%29%20ARRAY%20JOIN%20addr%3B'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/500


In [ ]:
async def test_query():
    q = "SELECT cutIPv6(probe_dst_prefix, 0, 0) as test, probe_dst_prefix FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604 LIMIT 10"
    with ff.TemporaryFile() as temp:
        with tqdm(desc="Running the query result", unit="MB") as pb:
            async for chunk in common.run_query(cfg=config, query=q):
                temp.write(chunk)
                pb.update(len(chunk) // config.chunk_size)
        temp.close()
        return pd.read_csv(temp.filename)
    
test_df = await test_query()
test_df